In [1]:
import torchaudio as ta
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd.profiler as profiler

# import pytorch_lightning as pl
import numpy as np
import os

import IPython.display as ipd

import numpy as np

import math

import glob

from tqdm.auto import tqdm

from python_files.Noise_Reduction_Datagen import Signal_Synthesis_DataGen

import warnings

warnings.filterwarnings("ignore")

import gc

# from numba import jit

In [2]:
torch.__version__

'1.6.0'

In [3]:
torch.cuda.is_available()

True

In [4]:
noise_dir = "./dataset/UrbanSound8K-Resampled/audio/"
signal_dir = "./dataset/cv-corpus-5.1-2020-06-22-Resampled/en/clips/"
signal_nums_save = "./dataset_loader_files/signal_paths_nums_save.npy"
num_noise_samples=100
num_signal_samples = 500
noise_save_path = "./dataset_loader_files/noise_paths_resampled_save.npy"
n_fft=400
win_length=n_fft
hop_len=n_fft//4
create_specgram = False
perform_stft = False
default_sr = 16000
sec = 4
augment=True
device_datagen = "cpu"

signal_synthesis_dataset = Signal_Synthesis_DataGen(noise_dir, signal_dir, \
                signal_nums_save=signal_nums_save, num_noise_samples=num_noise_samples, \
                num_signal_samples=num_signal_samples, noise_path_save=noise_save_path, \
                 n_fft=n_fft, win_length=win_length, hop_len=hop_len, create_specgram=create_specgram, \
                 perform_stft=perform_stft, normalize=True, default_sr=default_sr, sec=sec, augment=False, device=device_datagen)

Loading noise from saved file
Loading nums from npy file
500


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 64
shuffle = True
num_workers = 12
pin_memory = True
# data_loader = DataLoader(signal_synthesis_dataset, batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=num_workers)
data_loader = DataLoader(signal_synthesis_dataset, batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=num_workers, pin_memory=pin_memory)
# data_loader.to(device)

In [6]:
signal_synthesis_dataset.__len__()

50000

In [7]:
%%time
# data_loader_iter = iter(data_loader)
for index, i in enumerate(data_loader):
#     i = next(data_loader)
    if index < 32-1:
        pass
    else:
        break
    print(i[0].shape, index)

torch.Size([64, 64000]) 0
torch.Size([64, 64000]) 1
torch.Size([64, 64000]) 2
torch.Size([64, 64000]) 3
torch.Size([64, 64000]) 4
torch.Size([64, 64000]) 5
torch.Size([64, 64000]) 6
torch.Size([64, 64000]) 7
torch.Size([64, 64000]) 8
torch.Size([64, 64000]) 9
torch.Size([64, 64000]) 10
torch.Size([64, 64000]) 11
torch.Size([64, 64000]) 12
torch.Size([64, 64000]) 13
torch.Size([64, 64000]) 14
torch.Size([64, 64000]) 15
torch.Size([64, 64000]) 16
torch.Size([64, 64000]) 17
torch.Size([64, 64000]) 18
torch.Size([64, 64000]) 19
torch.Size([64, 64000]) 20
torch.Size([64, 64000]) 21
torch.Size([64, 64000]) 22
torch.Size([64, 64000]) 23
torch.Size([64, 64000]) 24
torch.Size([64, 64000]) 25
torch.Size([64, 64000]) 26
torch.Size([64, 64000]) 27
torch.Size([64, 64000]) 28
torch.Size([64, 64000]) 29
torch.Size([64, 64000]) 30
CPU times: user 3.48 s, sys: 1.26 s, total: 4.75 s
Wall time: 19 s


In [8]:
%%time
torch.stft(i[0][:72], n_fft=n_fft, hop_length=hop_len, win_length=win_length).shape

CPU times: user 21.7 ms, sys: 53.5 ms, total: 75.2 ms
Wall time: 74.7 ms


torch.Size([64, 201, 641, 2])

In [9]:
sig1 = i[0].unsqueeze(dim=1)
sig2 = i[0].unsqueeze(dim=1)
stacked_sig = torch.cat((sig1, sig2), dim=1)

In [10]:
sig2 = i[0].unsqueeze(dim=1)
sig2.shape

torch.Size([64, 1, 64000])

In [11]:
torch.sum(stacked_sig, dim=1).shape

torch.Size([64, 64000])

In [12]:
np.floor(((default_sr*sec) - (win_length - 1) - 1)/ hop_len + 3)

639.0

In [13]:
n_fft // 2 + 1

201

In [14]:
class InstantLayerNormalization(nn.Module):
    def __init__(self, in_shape, out_shape):
        self.in_shape = in_shape
        self.out_shape = out_shape
        
        self.epsilon = 1e-7
        self.gamma = None
        self.beta = None
        
        super(InstantLayerNormalization, self).__init__()
        
        self.gamma = torch.ones(out_shape)
        self.gamma = nn.Parameter(self.gamma)
        
        self.beta = torch.zeros(out_shape)
        self.beta = nn.Parameter(self.beta)
        
    def forward(self, inps):
        mean = inps.mean(-1, keepdim=True)
        variance = torch.mean(torch.square(inps - mean), dim=-1, keepdim=True)
        std = torch.sqrt(variance + self.epsilon)
        
        outs = (inps - mean) / std
        print(outs.shape, self.gamma.shape)
        outs = outs * self.gamma
        outs = outs + self.beta
        
        return outs
    
class Multiply():
    def __init__(self):
        super(Multiply, self).__init__()
    
    def forward(self, ten1, ten2):
        mul_out = torch.mul(ten1, ten2)
        return mul_out

class NoiseReducer(nn.Module):
    def __init__(self, default_sr, n_fft, win_length, hop_len, sec, dropout=0.5, batch_first=True, stride=2, normalized=False, bidir=False):
        
        self.default_sr = default_sr
        self.n_fft = n_fft
        self.win_length = win_length
        self.hop_len = hop_len
        self.sec = sec
        self.normalized = normalized
        
        # LSTM 1 UNITS
        self.rnn1_dims = n_fft // 2 + 1
        self.num_layers = 3
        self.batch_first = True
        self.dropout = 0.2
        self.bidir = bidir
        
        # LSTM 2 UNITS
        self.rnn2_dims = 4
        
        # Dense Layer Units
        self.inp_dims = int(np.floor(((default_sr*sec) - (win_length - 1) - 1)/ hop_len + 5))#3))

        super(NoiseReducer, self).__init__()
        
        self.lstm1 = nn.LSTM(input_size=self.rnn1_dims, hidden_size=self.rnn1_dims, num_layers=self.num_layers, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        self.lstm2 = nn.LSTM(input_size=self.rnn1_dims, hidden_size=self.rnn1_dims, num_layers=self.num_layers, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        
        print(self.rnn2_dims)
        self.lstm3 = nn.LSTM(input_size=self.rnn2_dims, hidden_size=self.rnn2_dims, num_layers=self.num_layers, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        self.lstm4 = nn.LSTM(input_size=self.rnn2_dims, hidden_size=self.rnn2_dims, num_layers=self.num_layers, batch_first=self.batch_first, dropout=self.dropout, bidirectional=self.bidir)
        
        
        self.instancenorm1 = nn.InstanceNorm1d(self.rnn1_dims)
        self.instancenorm2 = nn.InstanceNorm1d(self.rnn2_dims)
        
        self.dense1 = nn.Linear(self.rnn1_dims, self.rnn1_dims)
        self.dense2 = nn.Linear(self.rnn2_dims, self.rnn2_dims)
        
        self.conv1 = nn.Conv1d(1, self.rnn2_dims, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(self.rnn2_dims, self.rnn2_dims*2, kernel_size=3, stride=1, padding=1)
        
    @torch.jit.export
    def stft_layer(self, sig):
        
        sig_stft = torch.stft(sig, n_fft=self.n_fft, hop_length=self.hop_len, win_length=self.win_length)
        
        sig_cplx = torch.view_as_complex(sig_stft)
        mag = sig_cplx.abs().permute(0, 2, 1)
        angle = sig_cplx.angle().permute(0, 2, 1)
        
        return [mag, angle]
    
    @torch.jit.export
    def istft_layer(self, mag, angle):
        mag = mag.permute(0, 2, 1)
        angle = angle.permute(0, 2, 1)
        mag = torch.unsqueeze(mag, dim=-1)
        angle = torch.unsqueeze(angle, dim=-1)
        pre_stft = torch.cat((mag, angle), dim=-1)
        stft_sig = torch.istft(pre_stft, n_fft=self.n_fft, win_length=self.win_length, hop_length=self.hop_len)
        
        return stft_sig
    
    
    def forward(self, inp_tensor):
        
        mag, angle = self.stft_layer(inp_tensor)
        mag_norm = self.instancenorm1(mag)
        x, hidden_states = self.lstm1(mag_norm)
        x, hidden_states = self.lstm2(x, hidden_states)
        mask = self.dense1(x)
        mask = F.sigmoid(mask)
        estimated_mag = torch.mul(mag, mask)
        
        signal = self.istft_layer(estimated_mag, angle)
        signal = signal.unsqueeze(dim=1)

        feature_rep = self.conv1(signal)
        
        feature_norm = self.instancenorm2(feature_rep)
        
        feature_norm = feature_norm.permute(0, 2, 1)
        
        x, hidden_states = self.lstm3(feature_norm)
        x, hidden_states = self.lstm4(x, hidden_states)
        mask = self.dense2(x)
        feature_mask = F.sigmoid(mask)
        feature_mask = feature_mask.permute(0, 2, 1)
        estimate_feat = torch.mul(feature_rep, feature_mask)
        
        estimate_frames = self.conv2(estimate_feat)
        estimate_sig = torch.sum(estimate_frames, dim=1)
        
        return estimate_sig


In [15]:
scripted_model = torch.jit.script(NoiseReducer(default_sr, n_fft, win_length, hop_len, sec).to(device))

4


In [16]:
scripted_model.code

'def forward(self,\n    inp_tensor: Tensor) -> Tensor:\n  mag, angle, = (self).stft_layer(inp_tensor, )\n  mag_norm = (self.instancenorm1).forward(mag, )\n  _0 = (self.lstm1).forward__0(mag_norm, None, )\n  x, hidden_states, = _0\n  _1 = (self.lstm2).forward__0(x, hidden_states, )\n  x0, hidden_states0, = _1\n  mask = (self.dense1).forward(x0, )\n  mask0 = __torch__.torch.nn.functional.sigmoid(mask, )\n  estimated_mag = torch.mul(mag, mask0)\n  signal = (self).istft_layer(estimated_mag, angle, )\n  signal0 = torch.unsqueeze(signal, 1)\n  feature_rep = (self.conv1).forward(signal0, )\n  feature_norm = (self.instancenorm2).forward(feature_rep, )\n  feature_norm0 = torch.permute(feature_norm, [0, 2, 1])\n  _2 = (self.lstm3).forward__0(feature_norm0, None, )\n  x1, hidden_states1, = _2\n  _3 = (self.lstm4).forward__0(x1, hidden_states1, )\n  x2, hidden_states2, = _3\n  mask1 = (self.dense2).forward(x2, )\n  feature_mask = __torch__.torch.nn.functional.sigmoid(mask1, )\n  feature_mask0 = to

In [17]:
class Negative_SNR_Loss(nn.Module):
    def __init__(self):
        super(Negative_SNR_Loss, self).__init__()
    
    def forward(self, sig_pred, sig_true):
        
        sig_true_sq = torch.square(sig_true)
        sig_pred_sq = torch.square(sig_true - sig_pred)

        sig_true_mean = torch.mean(sig_true_sq)
        sig_pred_mean = torch.mean(sig_pred_sq)

        snr = sig_true_mean / sig_pred_mean + 1e-7
        loss = -1*torch.log10(snr)

        return loss

In [18]:
%%time
model = NoiseReducer(default_sr=default_sr, n_fft=n_fft, win_length=win_length, hop_len=hop_len, sec=sec).to(device)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = Negative_SNR_Loss()
n_epochs=100

model.train()
scaler = torch.cuda.amp.GradScaler()

4
CPU times: user 16.3 ms, sys: 2.08 ms, total: 18.3 ms
Wall time: 16.6 ms


In [19]:
fake_inputs = torch.randn(BATCH_SIZE, 96000).to(device)
with profiler.profile(profile_memory=True, record_shapes=False, use_cuda=True) as prof:
    model(fake_inputs)
print(prof.key_averages().table(sort_by="cuda_memory_usage"))

-------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------------------------------------  
Name                             Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     CUDA total %     CUDA total       CUDA time avg    CPU Mem          Self CPU Mem     CUDA Mem         Self CUDA Mem    Number of Calls  Input Shapes                                   
-------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------------------------------------  
empty                            0.12%            10.955ms         0.12%            10.955ms 

In [20]:
for epoch in range(1, n_epochs+1):
    loop = tqdm(enumerate(data_loader), leave=True, total=len(data_loader))
    train_loss = np.zeros((len(data_loader)))
    for index, (data, target) in loop:
        
        data = data.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
#         with torch.cuda.amp.autocast():
        output = scripted_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
                
        train_loss[index] = loss.item()
        if np.isnan(loss.item()) or np.isnan(np.sum(train_loss)/index+1e-5):
            print(f"Data shape = {data.shape}\nTarget Shape = {target.shape}")
        
        loop.set_description(f"Epoch: [{epoch}/{n_epochs}]\t")
        loop.set_postfix(loss = np.sum(train_loss)/index+1e-5)

KeyboardInterrupt: 

In [11]:
# del model
# gc.collect()
# torch.cuda.empty_cache()

In [21]:
noise_add_sig, main_sig = signal_synthesis_dataset.__getitem__(10000)
noise_add_sig = torch.unsqueeze(noise_add_sig, dim=0).to(device)
main_sig = torch.unsqueeze(main_sig, dim=0).to(device)

In [23]:
%%time
with torch.no_grad():
    outs = model(noise_add_sig)

CPU times: user 7.1 s, sys: 93.8 ms, total: 7.2 s
Wall time: 7.24 s


In [24]:
outs.shape

torch.Size([1, 64000])

In [25]:
x = outs[0].t().to("cpu").numpy()
ipd.Audio(x, rate=default_sr)

In [26]:
x = noise_add_sig[0].t().to("cpu").numpy()
ipd.Audio(x, rate=default_sr)

In [27]:
x = main_sig[0].t().to("cpu").numpy()
ipd.Audio(x, rate=default_sr)